# Analyze goodness of fit of behavioral measures with R
1. Run a separate within-subject GLM for each behavioral measure, with the behavioral measure included as a parametric modulator.
2. Compute Bayes information criterion (BIC) for each ROI/subject/measure.
  1. Extract residuals timeseries from ROIs for each subject/measure/run.
  2. Compute variance of residuals per voxel over the timeseries.
  3. Take logarithm of residual variance.
  4. Perform Kolmogorov-Smirnov test of normality of voxel-wise logarithmic residual variances.
    - Need more detail here. 
  5. Calculate BIC from logarithmic residual variances $log(\sigma ^{2})$, number of parameters in models *k*, and number of data points in models *n*:
$$ BIC = n*log(\sigma ^{2}) + k*log(n) $$
  6. Average BIC across voxels in ROI and runs per ROI/subject/measure.
3. Perform repeated measures ANOVA
  - IVs: measure, ROI
  - DV: BIC

In [ ]:
# Import relevant libraries
library('nlme')
library('multcomp')

In [ ]:
# Location of computed BIC/AIC values
f = 'repeated_anova_dset.csv'

# Read in data
myData = read.csv(file = f)

In [ ]:
# Set columns as factors
myData <- within(myData, {
  Subject <- factor(Subject)
  ROI <- factor(ROI)
  Measure <- factor(Measure)
})

# Sort by subject (unnecessary)
myData <- myData[order(myData$Subject), ]
head(myData)

In [ ]:
# Get mean across observations for each combo
# (unnecessary as BIC is average across ROI)
myData.mean <- aggregate(myData$BIC,
                         by=list(myData$Subject,
                                 myData$ROI,
                                 myData$Measure),
                         FUN='mean')
colnames(myData.mean) <- c("Subject", "ROI", "Measure", "BIC")
myData.mean <- myData.mean[order(myData.mean$Subject), ]
head(myData.mean)

In [ ]:
# Sort of from https://stats.stackexchange.com/a/13816
# and https://stats.stackexchange.com/a/23014
# and https://stats.stackexchange.com/a/10909
# but instead of two-way I'm doing four one-ways
nROIs = length(unique(myData.mean$ROI))
alpha = 0.05 / nROIs

for(i in unique(myData.mean$ROI)){
    print(i)
    redData = myData.mean[myData.mean$ROI==i,]
    model = lme(BIC ~ Measure, random=~1|Subject,
                data=redData)
    modelAnova = anova(model)
    print(modelAnova)
    p = modelAnova['Measure', 'p-value']
    if(p < alpha){
        print('ANOVA is significant. Performing post-hoc tests.')
        print(summary(glht(model, linfct=mcp(Measure="Tukey")),
                      test=adjusted(type="bonferroni")))
    }
    else{
        print('ANOVA is not significant. Skipping post-hoc tests.')
    }
}

## Plot means of combinations of ROI and Measure
Note that in the simulated dataset, the BIC values are random, but I shifted the means of Amygdala-Difficulty and ACC-Accuracy up by 0.2 so there'd be a couple of significant associations to find.

In [ ]:
with(myData, {interaction.plot(Measure, ROI, BIC,
                               col=c(1:4), lty=1,
                               ylab='Mean Bayesian Information Criterion',
                               trace.label='Region of Interest', 
                               xlab='Behavioral Measure',
                               main='Interaction Plot')})